In [4]:
# !pip install linformer
!pip install vit_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 850.0 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 1.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.8 MB/s eta 0:00:00


In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
from vit_pytorch.efficient import ViT
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
import torch.utils.data as data
import torchvision
from torchvision.transforms import ToTensor
torch.cuda.is_available()

True

In [2]:
# Hyperparameters:
batch_size = 32
epochs = 100
lr = 0.001
gamma = 0.7
seed = 142
IMG_SIZE = 200
patch_size = 20
num_classes = 2


In [3]:
train_ds = torchvision.datasets.ImageFolder("training_files/target_training_datasets/CHEMBL286/dataset/training", transform=ToTensor())
valid_ds = torchvision.datasets.ImageFolder("training_files/target_training_datasets/CHEMBL286/dataset/test", transform=ToTensor())
test_ds = torchvision.datasets.ImageFolder("training_files/target_training_datasets/CHEMBL286/dataset/validation", transform=ToTensor())

In [4]:
# Data Loaders:
train_loader = data.DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=4)
valid_loader = data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True,  num_workers=4)
test_loader  = data.DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=4)

/dgxa_home/se21ucse231/anaconda3/lib/python3.11/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [60]:
# Resize using transforms to 200

# Training device:
device = 'cuda'

# Linear Transformer:
efficient_transformer = Linformer(dim=128, seq_len=64, depth=12, heads=8, k=64)

# Vision Transformer Model: 
model = ViT(dim=128, image_size=200, patch_size=patch_size, num_classes=num_classes, transformer=efficient_transformer, channels=3).to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# Learning Rate Scheduler for Optimizer:
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [61]:
# Training:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0
    for data, label in tqdm(train_loader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

        with torch.no_grad():
            epoch_val_accuracy = 0
            epoch_val_loss = 0
            
        for data, label in valid_loader:
            
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )


  0%|          | 0/37 [00:00<?, ?it/s]

AssertionError: the sequence length of the key / values must be 64 - 101 given

In [10]:
# load saved model:
PATH = "epochs"+"_"+str(epochs)+"_"+"img"+"_"+str(IMG_SIZE)+"_"+"patch"+"_"+str(patch_size)+"_"+"lr"+"_"+str(lr)+".pt"
efficient_transformer = Linformer(dim=128, seq_len=49+1, depth=12, heads=8, k=64)
model = ViT(image_size=224, patch_size=32, num_classes=2, dim=128 ,transformer=efficient_transformer, channels=3)
model.load_state_dict(torch.load(PATH))

NameError: name 'epochs' is not defined

In [11]:
# Performance on Valid/Test Data
def overall_accuracy(model, test_loader, criterion):
    
    '''
    Model testing 
    
    Args:
        model: model used during training and validation
        test_loader: data loader object containing testing data
        criterion: loss function used
    
    Returns:
        test_loss: calculated loss during testing
        accuracy: calculated accuracy during testing
        y_proba: predicted class probabilities
        y_truth: ground truth of testing data
    '''
    
    y_proba = []
    y_truth = []
    test_loss = 0
    total = 0
    correct = 0
    for data in tqdm(test_loader):
        X, y = data[0].to('cpu'), data[1].to('cpu')
        output = model(X)
        test_loss += criterion(output, y.long()).item()
        for index, i in enumerate(output):
            y_proba.append(i[1])
            y_truth.append(y[index])
            if torch.argmax(i) == y[index]:
                correct+=1
            total+=1
                
    accuracy = correct/total
    
    y_proba_out = np.array([float(y_proba[i]) for i in range(len(y_proba))])
    y_truth_out = np.array([float(y_truth[i]) for i in range(len(y_truth))])
    
    return test_loss, accuracy, y_proba_out, y_truth_out


loss, acc, y_proba, y_truth = overall_accuracy(model, test_loader, criterion = nn.CrossEntropyLoss())


print(f"Accuracy: {acc}")

print(pd.value_counts(y_truth))

NameError: name 'model' is not defined

In [13]:
!pip install ipywidgets==8.0.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.2
    Uninstalling widgetsnbextension-3.5.2:
      Successfully uninstalled widgetsnbextension-3.5.2
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.6.5
    Uninstalling ipywidgets-7.6.5:
      Successfully uninstalled ipywidgets-7.6.5


In [14]:
import os
import numpy as np
import tensorflow as tf
from transformers import ViTFeatureExtractor, TFViTForImageClassification
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_dir = 'training_files/target_training_datasets/CHEMBL286/dataset/training'
val_dir = 'training_files/target_training_datasets/CHEMBL286/dataset/validation'
test_dir = 'training_files/target_training_datasets/CHEMBL286/dataset/test'

# Image size and batch size
img_size = 200
batch_size = 32

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Load ViT feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224-in21k', num_labels=2)

# Preprocessing function for ViT
def preprocess_images(images):
    images = [image.numpy() for image in images]
    images = np.stack(images, axis=0)
    images = feature_extractor(images, return_tensors='tf')['pixel_values']
    return images

# Data generator for ViT
def vit_data_generator(generator):
    while True:
        batch = next(generator)
        images = preprocess_images(batch[0])
        labels = batch[1]
        yield images, labels

# Prepare the dataset
train_dataset = tf.data.Dataset.from_generator(
    lambda: vit_data_generator(train_generator),
    output_signature=(
        tf.TensorSpec(shape=(None, 3, 224, 224), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: vit_data_generator(val_generator),
    output_signature=(
        tf.TensorSpec(shape=(None, 3, 224, 224), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

# Define the optimizer, loss, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.BinaryAccuracy()]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Evaluate the model
test_dataset = tf.data.Dataset.from_generator(
    lambda: vit_data_generator(test_generator),
    output_signature=(
        tf.TensorSpec(shape=(None, 3, 224, 224), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

results = model.evaluate(test_dataset, steps=len(test_generator))
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")


Found 1183 images belonging to 2 classes.
Found 297 images belonging to 2 classes.
Found 372 images belonging to 2 classes.


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing TFViTForImageClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTForImageClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'Variable' object has no attribute '_distribute_strategy'

In [3]:
# Binary Cross Entropy

In [6]:
# Install required packages
!pip install linformer
!pip install vit_pytorch

from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
from vit_pytorch.efficient import ViT
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
import torch.utils.data as data
import torchvision
from torchvision import transforms

# Check if CUDA is available
print(torch.cuda.is_available())

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Set seed for reproducibility
torch.manual_seed(142)

# Hyperparameters
batch_size = 32
epochs = 100
lr = 0.0001
gamma = 0.7
IMG_SIZE = 200
patch_size = 20
num_classes = 2

# Transforms for image resizing and normalization
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor()
])

# Define paths
train_dir = 'training_files/target_training_datasets/CHEMBL286/dataset/training'
val_dir = 'training_files/target_training_datasets/CHEMBL286/dataset/validation'
test_dir = 'training_files/target_training_datasets/CHEMBL286/dataset/test'

# Load datasets
train_ds = torchvision.datasets.ImageFolder(train_dir, transform=transform)
valid_ds = torchvision.datasets.ImageFolder(val_dir, transform=transform)
test_ds = torchvision.datasets.ImageFolder(test_dir, transform=transform)

# Data loaders
train_loader = data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = data.DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=0)

# Linear Transformer
efficient_transformer = Linformer(dim=128, seq_len=(IMG_SIZE // patch_size) ** 2 + 1, depth=12, heads=8, k=64)

# Vision Transformer Model
model = ViT(
    dim=128, 
    image_size=IMG_SIZE, 
    patch_size=patch_size, 
    num_classes=num_classes, 
    transformer=efficient_transformer, 
    channels=3
).to(device)

# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# Learning rate scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

# Training loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    epoch_accuracy = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(device), label.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)
    
    model.eval()
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in valid_loader:
            data, label = data.to(device), label.to(device)
            val_output = model(data)
            val_loss = criterion(val_output, label)
            
            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss / len(valid_loader)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, Val Loss: {epoch_val_loss:.4f}, Val Accuracy: {epoch_val_accuracy:.4f}")

# Save the model
PATH = f"epochs_{epochs}_img_{IMG_SIZE}_patch_{patch_size}_lr_{lr}.pt"
torch.save(model.state_dict(), PATH)

# Load saved model
efficient_transformer = Linformer(dim=128, seq_len=(IMG_SIZE // patch_size) ** 2 + 1, depth=12, heads=8, k=64)
model = ViT(image_size=IMG_SIZE, patch_size=patch_size, num_classes=num_classes, dim=128, transformer=efficient_transformer, channels=3).to(device)
model.load_state_dict(torch.load(PATH))

# Function to calculate overall accuracy
def overall_accuracy(model, test_loader, criterion):
    model.eval()
    y_proba = []
    y_truth = []
    test_loss = 0
    total = 0
    correct = 0
    with torch.no_grad():
        for data, label in tqdm(test_loader):
            data, label = data.to(device), label.to(device)
            output = model(data)
            test_loss += criterion(output, label.long()).item()
            for index, i in enumerate(output):
                y_proba.append(i[1].item())
                y_truth.append(label[index].item())
                if torch.argmax(i) == label[index]:
                    correct += 1
                total += 1
    accuracy = correct / total
    y_proba_out = np.array(y_proba)
    y_truth_out = np.array(y_truth)
    return test_loss, accuracy, y_proba_out, y_truth_out

# Evaluate model on test data
loss, acc, y_proba, y_truth = overall_accuracy(model, test_loader, criterion)

print(f"Test Accuracy: {acc:.4f}")

# Plot confusion matrix
cm = confusion_matrix(y_truth, np.argmax(y_proba.reshape(-1, 1), axis=1))
print(cm)


True


  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 1/100, Loss: 0.8648, Accuracy: 0.5545, Val Loss: 0.6675, Val Accuracy: 0.6122


  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 2/100, Loss: 0.6624, Accuracy: 0.6137, Val Loss: 0.6534, Val Accuracy: 0.6417


...

Epoch 99/100, Loss: 0.1047, Accuracy: 0.9721, Val Loss: 1.1589, Val Accuracy: 0.7073


  0%|          | 0/37 [00:00<?, ?it/s]

Epoch 100/100, Loss: 0.0658, Accuracy: 0.9848, Val Loss: 1.1144, Val Accuracy: 0.7184


  0%|          | 0/12 [00:00<?, ?it/s]

Test Accuracy: 0.6909
[[149   0]
 [223   0]]
